In [5]:
import yfinance as yf
import pandas as pd
import os

# Define a dictionary with indices and their corresponding ticker symbols
indices = {
    'S&P 500': '^GSPC',
    'Gold': 'GC=F',
    'ICE U.S. Dollar Index': '^NYICDX',
    'WTI Crude Oil': 'CL=F',
    'VIX index': '^VIX'
}

# Initialize a dictionary to store individual DataFrames
dataframes = {}

# Loop over each index to download data, process it, save as CSV, and prepare for merging
for index_name, ticker in indices.items():
    # Define the filename
    filename = f"{index_name.replace(' ', '_')}_closing_price_interpolated.csv"
    
    # Download data for the index
    try:
        data = yf.download(ticker, start='1927-12-30', end='2024-10-28')[['Open']].reset_index()
        data.columns = ['Date', 'Open_Price']

        # Convert date to datetime, set as index, and resample
        data = data.assign(Date=pd.to_datetime(data['Date'])).set_index('Date').resample('D').mean()

        # Interpolate missing values
        data['Open_Price'] = data['Open_Price'].interpolate(method='linear')

        # Reset index and save to CSV
        data.reset_index().to_csv(filename, index=False)
        print(f"File created: {filename}")

        # Rename the "Open_Price" column to the index name and store in dictionary
        df = data.reset_index()
        df = df.rename(columns={'Open_Price': index_name})
        dataframes[index_name] = df

    except Exception as e:
        print(f"Error processing {index_name}: {e}")

# Check if we have enough DataFrames to merge
if not dataframes:
    print("No data was successfully processed. Exiting.")
else:
    # Merge all DataFrames on the "Date" column
    merged_df = None
    for index_name, df in dataframes.items():
        if merged_df is None:
            merged_df = df  # Start with the first DataFrame
        else:
            merged_df = pd.merge(merged_df, df, on='Date', how='inner')  # Merge subsequent DataFrames
    
    # Save the merged DataFrame to a new CSV file
    merged_filename = "combined_data.csv"
    merged_df.to_csv(merged_filename, index=False)
    print(f"Merged data saved to {merged_filename}.")

print(merged_df)  # Display the last 10 rows of the merged DataFrame

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

File created: S&P_500_closing_price_interpolated.csv
File created: Gold_closing_price_interpolated.csv



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


File created: ICE_U.S._Dollar_Index_closing_price_interpolated.csv
File created: WTI_Crude_Oil_closing_price_interpolated.csv
File created: VIX_index_closing_price_interpolated.csv
Merged data saved to combined_data.csv.
           Date      S&P 500         Gold  ICE U.S. Dollar Index  \
0    2000-08-30  1509.839966   273.899994             112.190002   
1    2000-08-31  1502.589966   274.799988             111.980003   
2    2000-09-01  1517.680054   277.000000             112.360001   
3    2000-09-02  1518.452545   276.699997             112.172501   
4    2000-09-03  1519.225037   276.399994             111.985001   
...         ...          ...          ...                    ...   
8818 2024-10-21  5857.819824  2721.899902             103.459999   
8819 2024-10-22  5832.700195  2731.699951             103.970001   
8820 2024-10-23  5834.500000  2742.500000             104.089996   
8821 2024-10-24  5817.799805  2729.000000             104.430000   
8822 2024-10-25  5826.750000  2

In [6]:
import requests
import pandas as pd

# URL of the Excel file to download
url = "https://markets.newyorkfed.org/read?startDt=2016-03-01&endDt=2024-10-25&eventCodes=500&productCode=50&sort=postDt:-1,eventCode:1&format=xlsx"

# Send a GET request to download the Excel file
response = requests.get(url)

# Specify the local file path to save the downloaded Excel file
file_path = "../EFFR_data.xlsx"

# Save the Excel file locally
with open(file_path, 'wb') as f:
    f.write(response.content)

# Read the Excel file using pandas with the 'xlsxwriter' engine
df_EFFR = pd.read_excel(file_path)

# Convert 'Effective Date' column in df_EFFR to datetime
df_EFFR['Effective Date'] = pd.to_datetime(df_EFFR['Effective Date'])

# Select only the 'Effective Date' and 'Rate (%)' columns from df_EFFR
df_EFFR = df_EFFR[['Effective Date', 'Rate (%)']]

# Set the 'Effective Date' column as the index of df_EFFR
df_EFFR.set_index('Effective Date', inplace=True)

# Merge the "Rate (%)" column from df_EFFR to merged_df_step1 based on the date column
merged_df = merged_df.merge(df_EFFR, how='left', left_on='Date', right_index=True)

# Remove rows with NA values in the 'Rate (%)' column
merged_df = merged_df.dropna(subset=['Rate (%)'])

# Interpolate all columns in merged_df to fill in missing values
merged_df = merged_df.set_index('Date')
merged_df = merged_df.resample('D').interpolate(method='linear')
merged_df = merged_df.reset_index()
merged_df = merged_df.rename(columns={'index': 'Date'})

# Print the first few rows to verify the merge
print(merged_df)

/opt/network/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


           Date      S&P 500         Gold  ICE U.S. Dollar Index  \
0    2016-03-01  1937.089966  1240.500000              98.180000   
1    2016-03-02  1976.599976  1232.800049              98.260002   
2    2016-03-03  1985.599976  1238.400024              98.199997   
3    2016-03-04  1994.010010  1263.000000              97.669998   
4    2016-03-05  1994.710002  1261.933350              97.523333   
...         ...          ...          ...                    ...   
3156 2024-10-21  5857.819824  2721.899902             103.459999   
3157 2024-10-22  5832.700195  2731.699951             103.970001   
3158 2024-10-23  5834.500000  2742.500000             104.089996   
3159 2024-10-24  5817.799805  2729.000000             104.430000   
3160 2024-10-25  5826.750000  2725.500000             104.050003   

      WTI Crude Oil  VIX index  Rate (%)  
0         33.900002  19.840000      0.36  
1         33.889999  17.980000      0.37  
2         34.730000  17.250000      0.37  
3         3

In [7]:
# Data processing
from darts import TimeSeries
import pandas as pd
import numpy as np

# Function to calculate moving averages
def calculate_moving_averages(df, window):
    return df.rolling(window=window).mean()

# Calculate 3-day and 6-day moving averages for all columns except 'Date'
for col in merged_df.columns:
    if col not in ['Date']:
        merged_df[col + '_3D_MA'] = calculate_moving_averages(merged_df[col], window=3)
        merged_df[col + '_6D_MA'] = calculate_moving_averages(merged_df[col], window=6)

# Convert 'Date' column to datetime type and set it as index
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df.set_index('Date', inplace=True)

# Convert moving averages to binary variables
for col in merged_df.columns:
    if '_3D_MA' in col:
        base_col = col.replace('_3D_MA', '')
        merged_df[base_col + '_Binary'] = np.where(merged_df[col] > merged_df[base_col + '_6D_MA'], 1, 0)

# Remove original columns and moving average columns
columns_to_drop = [col for col in merged_df.columns if not col.endswith('_Binary')]
merged_df = merged_df.drop(columns=columns_to_drop)

# Print the updated DataFrame
print(merged_df.head(10))

            S&P 500_Binary  Gold_Binary  ICE U.S. Dollar Index_Binary  \
Date                                                                    
2016-03-01               0            0                             0   
2016-03-02               0            0                             0   
2016-03-03               0            0                             0   
2016-03-04               0            0                             0   
2016-03-05               0            0                             0   
2016-03-06               1            1                             0   
2016-03-07               1            1                             0   
2016-03-08               1            1                             0   
2016-03-09               0            0                             0   
2016-03-10               0            0                             0   

            WTI Crude Oil_Binary  VIX index_Binary  Rate (%)_Binary  
Date                                                 

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    log_loss, classification_report, roc_auc_score, confusion_matrix
)
from sklearn.model_selection import GridSearchCV
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, ClassifierMixin

# =======================
# 1. Data Split (80%/10%/10%)
# =======================
X = merged_df.drop(columns=['S&P 500_Binary']).copy()
y = merged_df['S&P 500_Binary'].copy()

print("Class distribution in the target variable:")
print(y.value_counts())

train_size = 0.8
val_size = 0.1
test_size = 0.1
n_total = len(X)

train_end = int(train_size * n_total)
val_end   = train_end + int(val_size * n_total)

X_train = X.iloc[:train_end].copy()
X_val   = X.iloc[train_end:val_end].copy()
X_test  = X.iloc[val_end:].copy()

y_train = y.iloc[:train_end].copy()
y_val   = y.iloc[train_end:val_end].copy()
y_test  = y.iloc[val_end:].copy()

# Combine train+val for walk-forward approach
X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Reshape for LSTM: (samples, seq_len=1, features)
X_train_val_np = X_train_val.values.reshape(
    (X_train_val.shape[0], 1, X_train_val.shape[1])
)
X_test_np = X_test.values.reshape(
    (X_test.shape[0], 1, X_test.shape[1])
)

# Convert to torch tensors
X_train_val_t = torch.tensor(X_train_val_np, dtype=torch.float32)
y_train_val_t = torch.tensor(y_train_val.values, dtype=torch.float32)
X_test_t      = torch.tensor(X_test_np,       dtype=torch.float32)
y_test_t      = torch.tensor(y_test.values,   dtype=torch.float32)

# =======================
# 2. LSTM Model
# =======================
class LSTMModel(nn.Module):
    """
    Returns raw logits (no final sigmoid).
    We'll apply sigmoid in predict/predict_proba().
    """
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers,
                 dropout_rate, activation, regularizer_strength):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout_rate
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.activation = getattr(torch.nn.functional, activation)
        self.regularizer_strength = regularizer_strength

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim, device=x.device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim, device=x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])  # last hidden state
        out = self.fc(out)                # raw logits
        return out

class PyTorchLSTM(BaseEstimator, ClassifierMixin):
    """
    Sklearn-style wrapper around the LSTM model.
    """
    def __init__(self,
                 input_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 dropout_rate,
                 activation,
                 regularizer_strength,
                 optimizer,
                 learning_rate,
                 batch_size,
                 epochs):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.regularizer_strength = regularizer_strength
        self.optimizer = optimizer
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        
        self.model = LSTMModel(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            n_layers=n_layers,
            dropout_rate=dropout_rate,
            activation=activation,
            regularizer_strength=regularizer_strength
        )
        self.criterion = nn.BCEWithLogitsLoss()
        self._optimizer = None

    def fit(self, X, y):
        self.model.train()
        self._optimizer = self._get_optimizer()
        
        dataset = torch.utils.data.TensorDataset(X, y)
        dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=self.batch_size, shuffle=True
        )
        
        for epoch in range(self.epochs):
            for inputs, labels in dataloader:
                self._optimizer.zero_grad()
                logits = self.model(inputs)
                loss = self.criterion(logits, labels.unsqueeze(1))
                loss.backward()
                self._optimizer.step()
        return self

    def predict_proba(self, X):
        self.model.eval()
        with torch.no_grad():
            logits = self.model(X)
            probs = torch.sigmoid(logits).cpu().numpy()
        return probs

    def predict(self, X):
        probs = self.predict_proba(X)
        return (probs >= 0.5).astype(float)

    def _get_optimizer(self):
        if self.optimizer == 'adam':
            return optim.Adam(self.model.parameters(), lr=self.learning_rate)
        elif self.optimizer == 'rmsprop':
            return optim.RMSprop(self.model.parameters(), lr=self.learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer}")

# =======================
# 3. Walk-forward + GridSearchCV
# =======================
window_size = 30
best_error = float('inf')
best_params = None

param_grid = {
    'batch_size': [2, 4, 8, 16],
    'hidden_dim': [32, 64, 128, 256, 512],
    'dropout_rate': [0.05, 0.1, 0.2],
    'activation': ['tanh', 'relu'],
    'regularizer_strength': [0.003, 0.005, 0.01, 0.015],
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [1e-6, 1e-5]
}

n_samples = len(X_train_val_t)
for start in range(0, n_samples - window_size, window_size):
    end = start + window_size
    test_end = end + window_size
    if test_end > n_samples:
        break

    X_train_fold = X_train_val_t[start:end]
    y_train_fold = y_train_val_t[start:end]
    X_val_fold   = X_train_val_t[end:test_end]
    y_val_fold   = y_train_val_t[end:test_end]
    
    # Skip if y_val_fold doesn't have both classes
    if len(torch.unique(y_val_fold)) < 2:
        print(f"Skipping fold [{start}:{end}] => val has single class.")
        continue

    # Make one single fold: train_idx (for X_train_fold), val_idx (for X_val_fold)
    foldX = torch.cat([X_train_fold, X_val_fold])
    foldY = torch.cat([y_train_fold, y_val_fold])
    
    train_idx = np.arange(len(X_train_fold))
    val_idx   = np.arange(len(X_train_fold), len(X_train_fold) + len(X_val_fold))
    single_fold = [(train_idx, val_idx)]
    
    # Base model with default hyperparams (just pick the first from each list)
    base_model = PyTorchLSTM(
        input_dim=X_train_val_t.shape[2],
        hidden_dim=param_grid['hidden_dim'][0],
        output_dim=1,
        n_layers=1,
        dropout_rate=param_grid['dropout_rate'][0],
        activation=param_grid['activation'][0],
        regularizer_strength=param_grid['regularizer_strength'][0],
        optimizer=param_grid['optimizer'][0],
        learning_rate=param_grid['learning_rate'][0],
        batch_size=param_grid['batch_size'][0],
        epochs=50
    )

    # Overriding default 5-fold with single_fold
    grid_search = GridSearchCV(
        estimator=base_model,
        param_grid=param_grid,
        scoring='neg_log_loss',
        n_jobs=-1,
        verbose=0,
        cv=single_fold,
        error_score='raise'  # raise an error if there's an invalid parameter combo
    )
    try:
        grid_search.fit(foldX, foldY)
    except Exception as e:
        print(f"GridSearch error in fold [{start}:{end}]: {e}")
        continue

    best_params_fold = grid_search.best_params_
    
    # Retrain with best params on the training fold
    best_model_fold = PyTorchLSTM(
        input_dim=X_train_val_t.shape[2],
        hidden_dim=best_params_fold['hidden_dim'],
        output_dim=1,
        n_layers=1,
        dropout_rate=best_params_fold['dropout_rate'],
        activation=best_params_fold['activation'],
        regularizer_strength=best_params_fold['regularizer_strength'],
        optimizer=best_params_fold['optimizer'],
        learning_rate=best_params_fold['learning_rate'],
        batch_size=best_params_fold['batch_size'],
        epochs=50
    )
    best_model_fold.fit(X_train_fold, y_train_fold)
    
    # Evaluate on val fold
    y_pred_proba_fold = best_model_fold.predict_proba(X_val_fold)
    fold_error = log_loss(y_val_fold, y_pred_proba_fold)
    
    if fold_error < best_error:
        best_error = fold_error
        best_params = best_params_fold

    print(f"Fold [{start}:{end}] -> LogLoss={fold_error:.4f}")

print(f"\nBest LogLoss overall: {best_error:.4f}")
print(f"Best params: {best_params}")

# =======================
# 4. Final evaluation on X_test
# =======================
# Train a final model with best_params on ALL training data
final_model = PyTorchLSTM(
    input_dim=X_train_val_t.shape[2],
    hidden_dim=best_params['hidden_dim'],
    output_dim=1,
    n_layers=1,
    dropout_rate=best_params['dropout_rate'],
    activation=best_params['activation'],
    regularizer_strength=best_params['regularizer_strength'],
    optimizer=best_params['optimizer'],
    learning_rate=best_params['learning_rate'],
    batch_size=best_params['batch_size'],
    epochs=50
)
final_model.fit(X_train_val_t, y_train_val_t)

y_pred_final = final_model.predict(X_test_t)
y_pred_proba_final = final_model.predict_proba(X_test_t)

print("\nFinal Classification Report:")
print(classification_report(y_test_t, y_pred_final))

print("\nFinal Confusion Matrix:")
print(confusion_matrix(y_test_t, y_pred_final))

auc_final = roc_auc_score(y_test_t, y_pred_proba_final)
print(f"\nArea Under Curve (AUC): {auc_final:.2f}")


Class distribution in the target variable:
S&P 500_Binary
1    1888
0    1273
Name: count, dtype: int64


/opt/network/anaconda3/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/opt/network/anaconda3/lib/python3.11/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/opt/network/anaconda3/lib/python3.11/site-packages/torch/autograd/__init__.py:

Error for fold starting at index 0: 0.69


/opt/network/anaconda3/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/opt/network/anaconda3/lib/python3.11/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/opt/network/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


Error for fold starting at index 30: 0.70


...

/opt/network/anaconda3/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/opt/network/anaconda3/lib/python3.11/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/opt/network/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


Error for fold starting at index 2760: 0.69

Best Error: 0.63
Best Parameters: {'activation': 'tanh', 'batch_size': 2, 'dropout_rate': 0.05, 'hidden_dim': 32, 'learning_rate': 1e-06, 'optimizer': 'adam', 'regularizer_strength': 0.003}


/opt/network/anaconda3/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "



Final Classification Report:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       108
         1.0       0.66      1.00      0.79       209

    accuracy                           0.66       317
   macro avg       0.33      0.50      0.40       317
weighted avg       0.43      0.66      0.52       317


Final Confusion Matrix:
[[  0 108]
 [  0 209]]

Area Under Curve (AUC): 0.81


/opt/network/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/network/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/network/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
